In [ ]:
%pip install "numpy<=2.2"
%pip install --upgrade meshio scipy
# Install compatible versions to avoid holoviews collections.Iterable issue
%pip install "holoviews>=1.15.0" "param>=1.12.0"
%pip install --upgrade uxarray rioxarray

import os
import sys
sys.path.append('.')
import meshio
import numpy as np
import xarray as xr
import uxarray as uxr
import rioxarray as xrio
from scipy.spatial import Delaunay

# On Docker turn off the warning on PROJ by specifying the PROJ lib path (uncomment the following line)
#os.environ['PROJ_LIB'] = '/opt/conda/envs/gospl/share/proj'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from scripts import umeshFcts as ufcts

# Create a replacement for meshplex functionality
class MeshTriAlternative:
    def __init__(self, vertices, cells):
        self.vertices = vertices
        self.cells = cells
        self.control_volumes = self._compute_control_volumes()
    
    def _compute_control_volumes(self):
        """Compute control volumes (areas) for each vertex using Voronoi-like approach"""
        n_vertices = len(self.vertices)
        volumes = np.zeros(n_vertices)
        
        # For each triangle, distribute 1/3 of its area to each vertex
        for cell in self.cells:
            # Get the three vertices of the triangle
            v0, v1, v2 = self.vertices[cell[0]], self.vertices[cell[1]], self.vertices[cell[2]]
            
            # Calculate triangle area using cross product
            area = 0.5 * abs(np.cross(v1 - v0, v2 - v0))
            
            # Distribute area equally to the three vertices
            for vertex_idx in cell:
                volumes[vertex_idx] += area / 3.0
        
        return volumes

# Monkey patch to replace meshplex
import sys
class MockMeshplex:
    @staticmethod
    def MeshTri(vertices, cells):
        return MeshTriAlternative(vertices, cells)

sys.modules['meshplex'] = MockMeshplex()

import matplotlib.pyplot as plt
%matplotlib inline

# Create a mesh from scratch

In cases where one to make some generic models, you could build your input file directly in many ways. 

Here is a simple example where we define a flat elevation at 100 m and a simple tectonic uplift (with a linear slope ranging to 5 mm/yr) and export it to a goSPL input file.

In [ ]:
dx = 200 # desired resolution
nx = 501 # desired number of nodes along the x-axis
ny = 501 # desired number of nodes along the y-axis
xdim = 1.0e5 # 100 km
ydim = 0.5e5 #  50 km
tmin = 0.
tmax = 0.005

xcoords = np.linspace(0,xdim,nx)
ycoords = np.linspace(0,ydim,ny)
X, Y = np.meshgrid(xcoords, ycoords)
# xcoords = np.arange(nx)*float(dx) 
# ycoords = np.arange(ny)*float(dx) 
# tecx = np.interp(xcoords, [xcoords[0],xcoords[-1]], [tmin,tmax])
# tec = np.broadcast_to(tecx, (nx, nx))[:ny,:]

elev = np.zeros_like(X)
# noise = np.random.normal(0, 0.05, tec.shape)
# elev = noise+100.

ds = xr.Dataset({
    'elev': xr.DataArray(
                data   = elev,
                dims   = ['y','x'],
                coords = {'x': xcoords,'y': ycoords},
    ),
    # 'tec': xr.DataArray(
    #             data   = tec,
    #             dims   = ['y','x'],
    #             coords = {'x': xcoords,'y': ycoords},
    #             )
    }
    )
ds['cellwidth'] = (['y','x'],dx*np.ones( (ny, nx)))
ds.elev.plot()
ds


We then build the unstructured grid for running goSPL

In [ ]:
output_path = "coupling_test" 
if not os.path.exists(output_path):
    os.makedirs(output_path)
    
# Build your planar mesh
ufcts.planarMesh(ds,output_path,fvtk='planar.vtk',fumpas=True,voro=True)


The mesh (`base2D.nc`) is now stored in the output folder (here named `coupling_test`). 

We will open this file and extract the information used in goSPL:

We now read the created `vtk` file and add the interpolated variables to it:

In [ ]:
mesh = meshio.read(output_path+'/planar.vtk')
vertex = mesh.points
cells = mesh.cells_dict['triangle']

### Creating goSPL input

We will now create the inputs for goSPL. We first start by creating the input mesh defining our UGRID structure:

In [ ]:
meshname = output_path+"/gospl_mesh"
np.savez_compressed(meshname, v=vertex, c=cells, 
                    z=np.zeros_like(vertex)
                    )